In [1]:
import os
import re
import tensorflow as tf
import numpy as np
import json
import collections
from sklearn.cross_validation import train_test_split
from pointer_generator_helper import (PointerGeneratorDecoder, 
                                      PointerGeneratorGreedyEmbeddingHelper, 
                                      PointerGeneratorBahdanauAttention,
                                      PointerGeneratorAttentionWrapper)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
labels = os.listdir('news')
news = ['news/' + i for i in labels if '.json' in i]
labels = [i.replace('.json','') for i in labels]
len(news)

123

In [3]:
import malaya
tokenizer = malaya.preprocessing._SocialTokenizer().tokenize

accept_tokens = ',-.()"\''

def is_number_regex(s):
    if re.match("^\d+?\.\d+?$", s) is None:
        return s.isdigit()
    return True

def detect_money(word):
    if word[:2] == 'rm' and is_number_regex(word[2:]):
        return True
    else:
        return False

def preprocessing(string):
    tokenized = tokenizer(string)
    tokenized = [w.lower() for w in tokenized if len(w) > 1 or w in accept_tokens]
    tokenized = ['<NUM>' if is_number_regex(w) else w for w in tokenized]
    tokenized = ['<MONEY>' if detect_money(w) else w for w in tokenized]
    return tokenized

def clean_label(label):
    string = re.sub('[^A-Za-z\- ]+', ' ', label)
    return re.sub(r'[ ]+', ' ', string.lower()).strip()

In [4]:
from sklearn.utils import shuffle
import random

min_len = 20
max_len = 500

x, y = [], []
for no, n in enumerate(news):
    with open(n) as fopen: 
        news_ = json.load(fopen)
    for row in news_:
        if len(row['text'].split()) > min_len:
            p = preprocessing(row['text'])
            if len(p) > max_len:
                p = random.sample(p, max_len)
            x.append(p)
            p = preprocessing(row['title'])
            y.append(p)
            
x, y = shuffle(x, y)

In [5]:
len(x), len(y)

(14471, 14471)

In [6]:
def build_dataset(words, n_words):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    count.extend(collections.Counter(words).most_common(n_words))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [7]:
import itertools

concat = list(itertools.chain(*x))
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])
print('filtered vocab size:',len(dictionary))
print("% of vocab used: {}%".format(round(len(dictionary)/vocabulary_size,4)*100))

vocab from size: 96677
Most common words [(',', 221350), ('.', 206810), ('yang', 76989), ('the', 76270), ('dan', 71493), ('-', 64992)]
Sample data [974, 290, 5, 2672, 267, 1234, 2660, 45333, 974, 809] ['azizah', 'sri', '.', 'patut', 'pakatan', 'persoalan', 'berjanji', 'kanlah', 'azizah', 'berikutan']
filtered vocab size: 96681
% of vocab used: 100.0%


In [8]:
for i in range(len(y)):
    y[i].append('EOS')

In [9]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [10]:
train_X, test_X, train_Y, test_Y = train_test_split(x, y, test_size = 0.2)

In [11]:
def sent2idx(sent, vocab, UNK=UNK):
    tokens = sent
    oovs = []
    extend_tokens = []
    tokenized = []
    for token in tokens:
        if token not in vocab:
            tokenized.append(UNK)
            if token not in oovs:
                oovs.append(token)
            extend_tokens.append(len(vocab) + oovs.index(token))
        else:
            extend_tokens.append(vocab[token])
            tokenized.append(vocab[token])
    return tokenized, extend_tokens, oovs

def target2idx(sent, oovs, vocab,UNK=UNK):
    tokens = sent
    tokenized = []
    for token in tokens:
        if token not in vocab:
            if token not in oovs:
                tokenized.append(UNK)
            else:
                tokenized.append(len(vocab) + oovs.index(token))
        else:
            tokenized.append(vocab[token])
    return tokenized

In [12]:
class Summarization:
    def __init__(self, size_layer, num_layers, embedded_size, dict_size):
        
        def lstm_cell(reuse=False):
            return tf.nn.rnn_cell.GRUCell(size_layer, reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        self.source_oov_words = tf.placeholder(tf.int32, shape=[])
        self.source_extend_tokens = tf.placeholder(tf.int32, shape=[None, None])
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        
        condition = tf.less(decoder_input, dict_size)
        self.decoder_input = self.Y
        self.decoder_input_length = self.Y_seq_len
        self.predict_count = tf.reduce_sum(self.decoder_input_length)
        
        embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(embeddings, self.X)
        encoder_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
        self.encoder_out, self.encoder_state = tf.nn.dynamic_rnn(cell = encoder_cells, 
                                                                 inputs = encoder_embedded, 
                                                                 sequence_length = self.X_seq_len,
                                                                 dtype = tf.float32)
        self.decode_initial_state = self.encoder_state[-1]
        print(self.decode_initial_state)
        
        atten_mech = PointerGeneratorBahdanauAttention(
                size_layer, self.encoder_out, memory_sequence_length=self.X_seq_len,
        coverage = True)
        decoder_cells = [lstm_cell() for _ in range(num_layers)]
        decoder_cells[0] = PointerGeneratorAttentionWrapper(
                cell=decoder_cells[0],
                attention_mechanism=atten_mech,
                attention_layer_size=size_layer,
                alignment_history = True,
                coverage = True
            )
        initial_state = [self.decode_initial_state for i in range(num_layers)]
        attention_cell_state = decoder_cells[0].zero_state(
                dtype=tf.float32, batch_size=batch_size)
        initial_state[0] = attention_cell_state.clone(
                cell_state=initial_state[0])
        self.initial_state = tuple(initial_state)
        decoder_cells = tf.contrib.rnn.MultiRNNCell(decoder_cells)
        
        decoded = tf.nn.embedding_lookup(embeddings, self.decoder_input)
        
        training_helper = tf.contrib.seq2seq.TrainingHelper(
            decoded,
            self.decoder_input_length
        )
        dense_layer = tf.layers.Dense(dict_size)
        
        training_decoder = PointerGeneratorDecoder(
            source_extend_tokens = self.source_extend_tokens,
            source_oov_words = self.source_oov_words,
            coverage = True,
            cell=decoder_cells,
            helper=training_helper,
            initial_state=self.initial_state,
            output_layer=dense_layer
        )
        
        maxlen = tf.reduce_max(self.decoder_input_length)
        train_dec_outputs, train_dec_last_state, _ = tf.contrib.seq2seq.dynamic_decode(
            training_decoder,
            output_time_major=False,
            impute_finished=True,
            maximum_iterations=maxlen,
            swap_memory=True)
        logits = train_dec_outputs.rnn_output
        
        masks = tf.sequence_mask(
            self.decoder_input_length, maxlen, 
            dtype=tf.float32)
        
        targets = tf.slice(self.Y, [0, 0], [-1, maxlen])
        i1, i2 = tf.meshgrid(tf.range(batch_size),
                     tf.range(maxlen), indexing="ij")
        indices = tf.stack((i1,i2,targets),axis=2)
        probs = tf.gather_nd(logits, indices)
        probs = tf.where(tf.less_equal(probs,0),tf.ones_like(probs)*1e-10,probs)
        crossent = -tf.log(probs)
        self.cost = tf.reduce_sum(crossent * masks) / tf.to_float(batch_size)
        alignment_history = train_dec_last_state[0].alignment_history.stack()
        alignment_history = tf.transpose(alignment_history,[1,2,0])
        coverage_loss = tf.minimum(alignment_history,tf.cumsum(alignment_history, axis=2, exclusive=True))
        self.coverage_loss = tf.reduce_sum(coverage_loss / tf.to_float(batch_size))
        self.cost = self.cost + self.coverage_loss
        
        self.optimizer = tf.train.AdamOptimizer().minimize(self.cost)
        
        helper = PointerGeneratorGreedyEmbeddingHelper(
            embedding=embeddings,
            start_tokens=tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
            end_token=EOS
        )
        
        inference_decoder = PointerGeneratorDecoder(
            source_extend_tokens = self.source_extend_tokens,
            source_oov_words = self.source_oov_words,
            coverage = True,
            cell=decoder_cells,
            helper=helper,
            initial_state=self.initial_state,
            output_layer=dense_layer
        )
        
        dec_outputs, dec_last_state, _ = tf.contrib.seq2seq.dynamic_decode(
            inference_decoder,
            output_time_major=False,
            maximum_iterations=tf.reduce_max(self.X_seq_len),
            swap_memory=True)
        
        self.beam_predictions = dec_outputs.sample_id
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        y_t = tf.argmax(logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:
size_layer = 128
num_layers = 2
embedded_size = 128
batch_size = 8

In [14]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Summarization(size_layer, num_layers, embedded_size, len(dictionary))
sess.run(tf.global_variables_initializer())

Tensor("rnn/while/Exit_4:0", shape=(?, 128), dtype=float32)
Tensor("decoder/while/PGDecoderStep/decoder/multi_rnn_cell/cell_0/cell_0/pointer_generator_attention_wrapper/Softmax:0", shape=(?, ?), dtype=float32) Tensor("decoder/while/PGDecoderStep/decoder/multi_rnn_cell/cell_0/cell_0/pointer_generator_attention_wrapper/Softmax:0", shape=(?, ?), dtype=float32)
Tensor("decoder/while/PGDecoderStep/decoder/multi_rnn_cell/cell_0/cell_0/pointer_generator_attention_wrapper/Softmax_1:0", shape=(?, ?), dtype=float32) Tensor("decoder/while/PGDecoderStep/decoder/multi_rnn_cell/cell_0/cell_0/pointer_generator_attention_wrapper/Softmax_1:0", shape=(?, ?), dtype=float32)


In [15]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [16]:
def batching(X, Y):
    s_, es_, oovs_, target_ = [], [], [], []
    for x, y in zip(X, Y):
        s,es,oovs = sent2idx(x, dictionary)
        target = target2idx(y, oovs,dictionary)
        s_.append(s)
        es_.append(es)
        oovs_.append(oovs)
        target_.append(target)
    s_ = pad_sequences(s_,padding='post')
    es_ = pad_sequences(es_,padding='post')
    target_ = pad_sequences(target_,padding='post')
    maxlen = max([len(o) for o in oovs_])
    return s_, es_, target_, maxlen

In [17]:
from tqdm import tqdm
from sklearn.utils import shuffle
import time

for EPOCH in range(10):
    lasttime = time.time()
    total_loss, total_accuracy, total_loss_test, total_accuracy_test = 0, 0, 0, 0
    train_X, train_Y = shuffle(train_X, train_Y)
    test_X, test_Y = shuffle(test_X, test_Y)
    pbar = tqdm(range(0, len(train_X), batch_size), desc='train minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(train_X))
        batch_x, batch_es, batch_y, maxlen = batching(train_X[k: index],
                                                     train_Y[k: index])
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                 model.source_extend_tokens:batch_es,
                                                model.Y:batch_y,
                                                model.source_oov_words:maxlen})
        total_loss += loss
        total_accuracy += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc='test minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(test_X))
        batch_x, batch_es, batch_y, maxlen = batching(test_X[k: index],
                                                     test_Y[k: index])
        acc, loss = sess.run([model.accuracy, model.cost], 
                                      feed_dict={model.X:batch_x,
                                                 model.source_extend_tokens:batch_es,
                                                model.Y:batch_y,
                                                model.source_oov_words:maxlen})
        total_loss_test += loss
        total_accuracy_test += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    total_loss /= (len(train_X) / batch_size)
    total_accuracy /= (len(train_X) / batch_size)
    total_loss_test /= (len(test_X) / batch_size)
    total_accuracy_test /= (len(test_X) / batch_size)
        
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(EPOCH, total_loss, total_accuracy))
    print('epoch: %d, avg loss test: %f, avg accuracy test: %f'%(EPOCH, total_loss_test, total_accuracy_test))

train minibatch loop:   0%|          | 0/1447 [00:00<?, ?it/s]

epoch: 0, avg loss: 36.730187, avg accuracy: 0.550886
epoch: 0, avg loss test: 20.387225, avg accuracy test: 0.772674


train minibatch loop:   0%|          | 0/1447 [00:00<?, ?it/s]

epoch: 1, avg loss: 16.800496, avg accuracy: 0.795760
epoch: 1, avg loss test: 16.749649, avg accuracy test: 0.826763


train minibatch loop:   0%|          | 0/1447 [00:00<?, ?it/s]

...

epoch: 7, avg loss: 4.132693, avg accuracy: 0.952237
epoch: 7, avg loss test: 12.061577, avg accuracy test: 0.895350


train minibatch loop:   0%|          | 0/1447 [00:00<?, ?it/s]

epoch: 8, avg loss: 4.318074, avg accuracy: 0.949479
epoch: 8, avg loss test: 11.472323, avg accuracy test: 0.903737


test minibatch loop: 100%|██████████| 362/362 [00:59<00:00,  6.09it/s, accuracy=0.684, cost=33.6]

epoch: 9, avg loss: 10.558175, avg accuracy: 0.885428
epoch: 9, avg loss test: 29.719049, avg accuracy test: 0.697240


In [19]:
batch_x, batch_es, batch_y, maxlen = batching(test_X[:1], test_Y[:1])

In [21]:
out = [rev_dictionary[i] for i in sess.run(model.beam_predictions, feed_dict = {model.X: batch_x,
                                             model.Y: batch_y,
                                             model.source_extend_tokens:batch_es,
                                             model.source_oov_words:maxlen})[0]]

In [22]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return ' '.join([x for x in seq if not (x in seen or seen_add(x))])

In [23]:
f7(out)

'emas UNK'

In [24]:
test_Y[:1]

[['ops', 'perdana', '<NUM>', 'penagih', 'dadah', 'diberkas', 'aadk', 'EOS']]